In [6]:
import requests
import os
import json
import sys
import findspark
findspark.init()

from pyspark.sql import SparkSession,Row
spark = SparkSession.builder.appName('TweetStreamer').getOrCreate()

In [7]:
take_input=input("What kind of streaming you want? IntervalBased/CountBased: ")

list_save=[]

bearer_token = "AAAAAAAAAAAAAAAAAAAAAH24jwEAAAAAz3dVqN2z83r%2FG2tIFPqD%2BGmDfJY%3Db5NcEA6yNGxJ1A6dFfqPoiLRFGwLii0kZLimmGegdKyeZVM3eJ"
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    #print("R: ",r)
    return r

What kind of streaming you want? IntervalBased/CountBased: IntervalBased


In [8]:
r={"Authorization":f"Bearer {bearer_token}", "User-Agent":"v2FilteredStreamPython"}


response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
print(json.dumps(response.json()))
rules=response.json()
ids = list(map(lambda rule: rule["id"], rules["data"]))
payload = {"delete": {"ids": ids}}

{"data": [{"id": "1603090960841220096", "value": "wtf lang:en -is:retweet"}], "meta": {"sent": "2022-12-14T18:37:42.157Z", "result_count": 1}}


In [12]:

response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )

sample_rules = [
        {"value": "RSS lang:en -is:retweet"}
        
    ]
payload = {"add": sample_rules}

response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )


In [13]:
from textblob import TextBlob
tweet = ""

if take_input=="IntervalBased":
          #start = time.time()
          import time
          period=float(input("Enter Interval in mins: "))
          period*=60
          print("PERIOD",period," SEC")
          start = time.time()
else:
          total_tweets=int(input("How many tweets you want: "))
          num=1

response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,timeout=100
    )
print(response.status_code)
if response.status_code != 200:
    raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            #print(json.dumps(json_response, indent=4, sort_keys=True))
            print("------------------")
#             print(json_response['data']['text'])
            list_save.append(json_response['data']['text'])
            tweet = json_response['data']['text']
            print(tweet)
            data = TextBlob(tweet)
            print(data.sentiment)
            if data.sentiment.polarity > 0:
                print("This is a POSITIVE tweet")
            if data.sentiment.polarity < 0:
                print("This is a NEGATIVE tweet")
            if data.sentiment.polarity == 0:
                print("This is a NEUTRAL tweet")
            print("------------")
            print("\n")
            if take_input=="IntervalBased":
                          if time.time() > start + period:
                 
                              break
            else:
                     num+=1
                     if num>total_tweets:
                            break

response.close()
print("connection closed")
# print(list_save)
sparkContext=spark.sparkContext
rdd=sparkContext.parallelize(list_save)
rddCollect = rdd.collect()
print("Number of Partitions: "+str(rdd.getNumPartitions()))
# print(rddCollect)

Enter Interval in mins: 2
PERIOD 120.0  SEC
200
------------------
@DailyLoud Oh come on people, asking if this is real..I bet you would stick your head in a fire if I told you you could see hell. Like someone's driveway is that high over an interstate...just good video editing..
Sentiment(polarity=0.43, subjectivity=0.5700000000000001)
This is a POSITIVE tweet
------------


------------------
@saotolover GOOD
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
This is a POSITIVE tweet
------------


------------------
WTF can you say to this level of stupidity? https://t.co/yAoh431uXJ
Sentiment(polarity=-0.55, subjectivity=1.0)
This is a NEGATIVE tweet
------------


------------------
@kittygangpjmn Wtf is that LOL
Sentiment(polarity=0.15000000000000002, subjectivity=0.85)
This is a POSITIVE tweet
------------


------------------
@spookyskelekat wtf! this deserves so many more likes, awesome work!
Sentiment(polarity=0.34375, subjectivity=0.75)
This is a POSITIVE tweet
--------

------------------
STOP TAGGING ME IN MARKET STUFF
Sentiment(polarity=0.0, subjectivity=0.0)
This is a NEUTRAL tweet
------------


------------------
@RaffaelS1337 gol france
Sentiment(polarity=0.0, subjectivity=0.0)
This is a NEUTRAL tweet
------------


------------------
@POOCHDOGGG It happens everytime you come back on the game I made all my classes again then had to get off and when I started the game back up they were all gone again
Sentiment(polarity=-0.2, subjectivity=0.2)
This is a NEGATIVE tweet
------------


------------------
Boufal i love you but wtf
Sentiment(polarity=0.0, subjectivity=0.8)
This is a NEUTRAL tweet
------------


------------------
when ppl copy me it gets me so annoyed like be urself bro wtf the amount of friends i use to have that copied me is crazy i would let it slide but it always made me look at them different
Sentiment(polarity=-0.375, subjectivity=0.8250000000000001)
This is a NEGATIVE tweet
------------


------------------
PLEASE LET PNG DIE AL

------------------
@JoelMetsNY Allegedly it’s because of her stuff that got leaked was racier than they expected
Sentiment(polarity=-0.1, subjectivity=0.25)
This is a NEGATIVE tweet
------------


------------------
@alifarhat79 You put money on it, didn’t you?
Sentiment(polarity=0.0, subjectivity=0.0)
This is a NEUTRAL tweet
------------


------------------
@Publius35818007 @Dude_WTF_Bro @bennyjohnson DEFINITELY not a cult https://t.co/5CDnyAmurk
Sentiment(polarity=0.0, subjectivity=0.5)
This is a NEUTRAL tweet
------------


------------------
Lol wtf is this game
Sentiment(polarity=-0.033333333333333326, subjectivity=0.7000000000000001)
This is a NEGATIVE tweet
------------


------------------
@GWilson2105 @NoContextBrits I spotted that shit! Wtf! No English man would touch a fake aldi toffee crisp!
Sentiment(polarity=-0.2125, subjectivity=0.6433333333333333)
This is a NEGATIVE tweet
------------


------------------
I have yet to receive my twitch recap wtf
Sentiment(polarity=-0.

------------------
@RapSheet Automatic ejection? Wtf no. Reviews would be nice tho
Sentiment(polarity=0.04999999999999999, subjectivity=1.0)
This is a POSITIVE tweet
------------


------------------
@nickelsgb Happy Christmas Brighton University
Sentiment(polarity=0.8, subjectivity=1.0)
This is a POSITIVE tweet
------------


------------------
@TummaKaru Omfg I been looking for it since the beginning of this summer! Also that value Sriracha is pretty good
Sentiment(polarity=0.475, subjectivity=0.8)
This is a POSITIVE tweet
------------


------------------
that moroccan defender dude was so adorable wtf
Sentiment(polarity=0.0, subjectivity=1.0)
This is a NEUTRAL tweet
------------


------------------
nah france is winning this one
Sentiment(polarity=0.5, subjectivity=0.75)
This is a POSITIVE tweet
------------


------------------
@dalisalx3 that’s what it reminded me of at first! facebook statuses lol i swear they are fresh out of ideas 😂
Sentiment(polarity=0.4708333333333334, subj

------------------
@TonyNashNerd No, all you have to do is take a look in the comments and you can't help but be disappointed - also, the general unwillingness to show PnLs doesn't testify to congruency between opinions expressed and results achieved...
Sentiment(polarity=-0.35, subjectivity=0.625)
This is a NEGATIVE tweet
------------


------------------
@AnaThePsychic_ It’s Instagram, I feel so scared right now I can’t stop shaking and crying 

I don’t know what to do
Sentiment(polarity=0.042857142857142844, subjectivity=0.5678571428571428)
This is a POSITIVE tweet
------------


------------------
I read the entire thread and I still don't know WTF this is about??? https://t.co/P27DmKt7f7
Sentiment(polarity=-0.25, subjectivity=0.8125)
This is a NEGATIVE tweet
------------


------------------
@Juliest101 He’s a just a thug in a suit.
Sentiment(polarity=0.0, subjectivity=0.0)
This is a NEUTRAL tweet
------------


------------------
@BBigDanzy Cool one
Sentiment(polarity=0.35, subje

------------------
@GetFuktHoosier Soak it in hot Epsom to take the inflammation down. 

Ingrowns are no joke!
Sentiment(polarity=0.027777777777777762, subjectivity=0.5694444444444445)
This is a POSITIVE tweet
------------


------------------
@Kabbageclip @kareemfanpage @sizziiii Wtf you mean tag team 💀
Sentiment(polarity=-0.40625, subjectivity=0.84375)
This is a NEGATIVE tweet
------------


------------------
@playmatejaylene @Daff_thinginab @RASC_WTF Thank you so much! I'm glad you like 🥰
Sentiment(polarity=0.5, subjectivity=1.0)
This is a POSITIVE tweet
------------


------------------
Anyways I got 13 https://t.co/yoR2vgUJVR
Sentiment(polarity=0.0, subjectivity=0.0)
This is a NEUTRAL tweet
------------


------------------
Patrick Dempsey's New Buzzed Hair Transformation Is Something Out of Your McDreams 
https://t.co/qqC084yRqg
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
This is a POSITIVE tweet
------------


------------------
New Video Up Now!
G

------------------
@cheapsuperman @Paper_hands92 You’ll unsderstand when you reach our age young superman.
Sentiment(polarity=0.1, subjectivity=0.4)
This is a POSITIVE tweet
------------


------------------
@caitlinkoi_ huh this isn't a photo wtf
Sentiment(polarity=-0.5, subjectivity=1.0)
This is a NEGATIVE tweet
------------


------------------
@Juliest101 Roid Rage 😡😡😡😡
Sentiment(polarity=0.0, subjectivity=0.0)
This is a NEUTRAL tweet
------------


------------------
@f_rose_v WHAT
Sentiment(polarity=0.0, subjectivity=0.0)
This is a NEUTRAL tweet
------------


------------------
@zaiavibes @raphousetv2 Right
Sentiment(polarity=0.2857142857142857, subjectivity=0.5357142857142857)
This is a POSITIVE tweet
------------


------------------
@_chinabosslady Go to yo dms on ig you'll see em at the top mfs just talking they shit 😂
Sentiment(polarity=0.15, subjectivity=0.65)
This is a POSITIVE tweet
------------


------------------
@DavidUlloaCuba One magic tool to remember is Blockchai

------------------
@DanielTurnerPTF Wtf?! Can he just go take his dirt nap already!! Evil fuck!!🤬🤬🤬🤬
Sentiment(polarity=-0.98828125, subjectivity=1.0)
This is a NEGATIVE tweet
------------


------------------
WTF!! How can you steal the video and make it yours?? DELETE IT https://t.co/s8G1xKbRxq
Sentiment(polarity=-0.78125, subjectivity=1.0)
This is a NEGATIVE tweet
------------


------------------
LORD HELP ME IM ON THIS NIGGA BAD WTF
Sentiment(polarity=-0.5999999999999999, subjectivity=0.8333333333333333)
This is a NEGATIVE tweet
------------


------------------
Webbie really had on some NIKE cargos 😭😭😭😭 man wtf
Sentiment(polarity=-0.15, subjectivity=0.6)
This is a NEGATIVE tweet
------------


------------------
@tchouameniszn @Ultra_Suristic He literally swings his right foot in a tackle movement wtf are you talking about there is no way someone puts his foot like that for no reason but to tackle
Sentiment(polarity=-0.10714285714285715, subjectivity=0.7678571428571428)
This is a

------------------
If anyone takes my theory I’m beating their ass wtf DONT STEAL MY THOUGHTS
Sentiment(polarity=-0.5, subjectivity=1.0)
This is a NEGATIVE tweet
------------


------------------
Wtf Fubo
Sentiment(polarity=-0.5, subjectivity=1.0)
This is a NEGATIVE tweet
------------


------------------
@elbitchin 🤣🤣🤣 you do not know what an aibo is???
Sentiment(polarity=0.0, subjectivity=0.0)
This is a NEUTRAL tweet
------------


------------------
@JLo gurl can you like respond??? Cause wtf?? https://t.co/IbQLtuG4Np
Sentiment(polarity=-0.5, subjectivity=1.0)
This is a NEGATIVE tweet
------------


------------------
Wtf is going on @fuboTV???
Sentiment(polarity=-0.5, subjectivity=1.0)
This is a NEGATIVE tweet
------------


------------------
wait wtf i tweeted gn twice am i aging i genuinleyvdont remember tweeting the first ojne wtf
Sentiment(polarity=-0.25, subjectivity=0.7777777777777777)
This is a NEGATIVE tweet
------------


------------------
@ImTruthy @AshleyDCan HAHAHAHA 